In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import root
import yfinance as yf
import matplotlib.pyplot as plt


In [ ]:
def merton_equations(x, E, sigma_E, D, r, T):
    V, sigma_V = x
    d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
    d2 = d1 - sigma_V * np.sqrt(T)
    eq1 = V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E
    eq2 = sigma_V * V * norm.cdf(d1) - sigma_E * E
    return [eq1, eq2]


In [ ]:
# import yfinance as yf
# import pandas as pd

# def get_price_and_vol(ticker, period="1y"):
#     try:
#         stock = yf.Ticker(ticker)
#         hist = stock.history(period=period)

#         if hist.empty:
#             raise ValueError("Données historiques vides.")

#         # Nettoyage
#         hist = hist.dropna()
#         adj_close = hist["Close"]  # ou "Adj Close" si présent
#         price = adj_close.iloc[-1]
#         returns = adj_close.pct_change().dropna()
#         sigma = returns.std() * (252 ** 0.5)

#         return price, sigma, hist

#     except Exception as e:
#         print(f"Erreur pour {ticker} : {e}")
#         return None, None, None


In [ ]:
# price, sigma_E, df = get_price_and_vol("AAPL")
# print(f"Prix actuel : {price:.2f}, Vol annualisée : {sigma_E:.2%}")


In [ ]:
# Télécharger les données
ticker = "AAL"
data = yf.download(ticker, period="1y")

# Vérifier si les données sont bien présentes
if data.empty:
    raise ValueError(f"Aucune donnée disponible pour le ticker {ticker}.")

# Choisir la bonne colonne de prix
price_col = 'Adj Close' if 'Adj Close' in data.columns else 'Close'
prices = data[price_col].dropna()

# Prix actuel de l'action (dernier jour)
price = float(prices.iloc[-1].item())

# Calcul des rendements log
returns = np.log(prices / prices.shift(1)).dropna()

# Volatilité annualisée de l'action (σ_E)
sigma_E = float(returns.std()) * np.sqrt(252)


# Paramètres Merton
E = price * 660_000_000        # Capitalisation boursière
D =40_000_000_000              # Dette (valeur nominale)
T = 1.0                         # Horizon 1 an
r = 0.04                        # Taux sans risque

# Affichage


print(f"Prix actuel : {price:.2f} USD")
print(f"Volatilité annualisée : {sigma_E:.2%}")
print(f"Capitalisation estimée : {E/1e9:.2f} Bn USD")


In [ ]:
initial_guess = [E + D, 0.3]
sol = root(merton_equations, initial_guess, args=(E, sigma_E, D, r, T))
V, sigma_V = sol.x


In [ ]:
d2 = (np.log(V / D) + (r - 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
PD = norm.cdf(-d2)
print(f"Probabilité de défaut estimée : {PD:.2%}")


In [ ]:

ticker = "AAL"
stock = yf.Ticker(ticker)
info = stock.info

shares = info.get("sharesOutstanding")
debt = info.get("totalDebt")

print(f"Shares outstanding: {shares:,}")
print(f"Total debt: {debt:,} USD")